In [2]:
import cv2
import tensorflow as tf
from sklearn import datasets, model_selection, metrics
import trimesh
import open3d as o3d
import numpy as np
from tensorflow import keras

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
import os

# Pedir el nombre del objeto
object_name = input("Introduce el nombre del objeto: ")

# Crear la carpeta si no existe
if not os.path.exists(object_name):
    os.makedirs(object_name)

# Iniciar la cámara
cap = cv2.VideoCapture(0)

image_count = len(os.listdir(object_name))  # Contar el número de imágenes existentes en la carpeta

while True:
    # Capturar frame por frame
    ret, frame = cap.read()
    
    # Mostrar el frame resultante
    cv2.imshow('Frame', frame)
    
    # Verificar si se presionó la tecla 'e' para guardar las imágenes
    if cv2.waitKey(1) & 0xFF == ord('e'):
        for i in range(15):  # Tomar 5 fotos
            ret, frame = cap.read()
            cv2.imwrite(f"{object_name}/image_{image_count}.jpg", frame)
            image_count += 1
    
    # Salir con la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la cámara y cerrar todas las ventanas
cap.release()
cv2.destroyAllWindows()


In [21]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Función para cargar imágenes desde una carpeta
def load_images_from_folder(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
            labels.append(folder)
    return images, labels

# Cargar imágenes y etiquetas
images, labels = load_images_from_folder(folder_name)

# Convertir listas a arrays de numpy
images = np.array(images)
labels = np.array(labels)

# Preprocesar las imágenes
images = images / 255.0  # Normalizar las imágenes

# Codificar las etiquetas
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Crear el modelo
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(images.shape[1], images.shape[2], images.shape[3])),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(lb.classes_), activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Guardar el modelo
model.save('object_recognition_model.h5')


Epoch 1/10
3/3 [==============================] - 15s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/10
3/3 [==============================] - 13s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
3/3 [==============================] - 12s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
3/3 [==============================] - 12s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
3/3 [==============================] - 11s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/10
3/3 [==============================] - 11s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/10
3/3 [==============================] - 11s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 -

In [31]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Cargar el modelo
model = load_model('object_recognition_model.h5')

# Iniciar la cámara
cap = cv2.VideoCapture(0)

# Función para preprocesar la imagen
def preprocess_image(image):
    resized_image = cv2.resize(image, (640, 480))  # Redimensionar la imagen a (480, 640)
    preprocessed_image = resized_image / 255.0  # Normalizar la imagen
    preprocessed_image = np.expand_dims(preprocessed_image, axis=0)  # Agregar una dimensión adicional para el batch
    return preprocessed_image

while True:
    # Capturar frame por frame
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # Preprocesar la imagen
    preprocessed_image = preprocess_image(frame)
    
    # Realizar la predicción
    prediction = model.predict(preprocessed_image)
    predicted_class = np.argmax(prediction, axis=1)
    predicted_label = lb.classes_[predicted_class[0]]
    
    # Convertir la imagen a escala de grises y detectar bordes
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    
    # Encontrar contornos
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Dibujar contornos en la imagen original
    cv2.drawContours(frame, contours, -1, (0, 255, 0), 2)
    
    # Mostrar la etiqueta predicha en el frame
    cv2.putText(frame, f'Predicted: {predicted_label}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    
    # Mostrar el frame resultante
    cv2.imshow('Frame', frame)
    
    # Salir con la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la cámara y cerrar todas las ventanas
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 79ms/step
